<a href="https://colab.research.google.com/github/adrijanailic/GI_projekat/blob/master/determine_variant2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import collections
import re
import numpy as np

def ideal_rr_count(read_result):
    # read_result je string pa mogu odmah da promijenim , u .
    # i mala slova u velika
    read_result = read_result.replace(',','.').upper()
    char_counter = collections.Counter(read_result).most_common() # Prebroji karaktere i poreda ih od najcescih do najrijedjih
    '''
    match = 0
    alt_A = 0
    alt_G = 0
    alt_T = 0
    alt_C = 0
    for item in m_c:
      if "." or "," in item:
        match += item[1]
      elif "A" or "a" in item:
        alt_A += item[1]
      elif "G" or "g" in item:
        alt_G += item[1]
      elif "T" or "t" in item:
        alt_T += item[1]
      elif "C" or "c" in item:
        alt_C += item[1]
      elif "$" in item:
        continue
      elif "*" in item:
        continue
    # Da li trebam brojati zvjezdice???
    var_list = [('match',match),('alt_A',alt_A),('alt_G',alt_G),('alt_T',alt_T),('alt_C',alt_C)]'''
    var_list = char_counter 
    return var_list

# Funkcija za izdvajanjanje insercija ili delecija ili oboje istovremeno, iz originalnog stringa read_result
# Povratna vrijednost su dvije lista, prva lista je lista tacaka, zareza i mozda drugih karaktera
# Druga lista je lista insercija i delecija
def segregate_indels(read_result):
  # Iskoristila sam re, da na laksi nacin pronadjem brojeve, jer su oni zajednicki za insercije i delecije
  # Mozda je ovo moguce napraviti i tako sto se prvo read result iz stringa prebaci u array, ali da se pripazi na visecifrene brojeve
  # Pa da se iskoristi np.findall ili slicno da se nadju ti brojevi, i njihovi indeksi i na slican nacin kao u ovom kodu, izbace odgovarajuci el iz tog arraya
  numbers = np.array([int(m) for m in re.findall('[\d]+', read_result)])
  # Indeksi brojeva
  indices = np.array([m.span() for m in re.finditer('[\d]+', read_result)])
  # String -> np.array
  read_result = np.array([char for char in read_result])
  # Formiram listu uredjenih dvojki kojih ima koliko i indelsa, a prvi broj u toj dvojki odgovara pocetnom indeksu insercije, tj tacki ili zarezu, 
  # A drugi broj odgovara indeksu na kom se nalazi zadnji element te insercije
  ranges = [(index[0]-2,index[1]+numbers[i]) for i,index in enumerate(indices)]
  # Formiranje liste insercija na koju je moguce primjeniti collection.counter
  indels_list = []
  for item in ranges:
    indels_list.append(''.join(read_result[item[0]:item[1]]))

  # Brisanje insercija iz read_results
  indices_for_removing = []
  for item in ranges:
    indices_for_removing.extend([*range(item[0],item[1])])
  read_result = np.delete(read_result,indices_for_removing)  

  return read_result, indels_list

def delete_carets(read_result):
  # Pomocu re nadjem sve indekse u read_results gdje se nalazi kapica
  indices = np.array([m.start() for m in re.finditer('\^', read_result)])
  # Azuriraj listu indeksa, jer je potrebno ukloniti i onaj indeks poslije indeksa kapice
  indices = np.concatenate((indices, indices+1))
  read_result = np.array([char for char in read_result])
  read_result = np.delete(rr_list, indices)  
  return read_result

def determine_variant(read_result):

  # Ovo je prva opcija, znaci da koristim find koja ce samo da posluzi za flag
  ins = read_result.find("+") # Flag za inserciju
  dele = read_result.find("-") # Flag za deleciju
  car = read_result.find("^") # Flag za caret
  # Druga opcija je da koristim re bibl da nadjem odmah i indekse svih insercija, delecija i kapica, kako bih onda mogla da ih izbacim
  # Zasad ostavljam prvu opciju
  
  if ins == -1 and dele == -1 and car == -1:
    print("No insertion, deletions or carets")
    list_of_vars = ideal_rr_count(read_result)
    # TO DO
    Varijanta = ## TO DO

  # Sljedeci slucaj je da nemamo ni insercija ni delecija samo kapicu imamo, izbrisemo kapicu i znak pored, i primjenimo ovo gore
  elif (car != -1) and (ins == -1 and dele == -1):
    print("Carets yes, no indels")
    read_result = delete_carets(read_result)
    # Prakticno smo dobili idealan read result i mozemo pozvati funkciju ideal_rr_count
    list_of_vars = ideal_rr_count(read_result)

    Varijanta = ## TO DO

  # Ukoliko imam insercije ili delecije, ili oboje istovremeno
  elif (ins != -1) or (dele != -1):
    # Provjeri ima li kapica
    if car != -1:
      print("Carets! Oh, no!!!")
      read_result = delete_carets(read_result)

    print("No carets")
    read_result, indels_list = segregate_indels(read_result)

    # zarezi -> tacke, mala slova -> VELIKA SLOVA
    indels_list = [s.replace(',','.') for s in indels_list]
    number_of_insertions = collections.Counter(map(str.upper, indels_list).most_common()
    # Pozvati ideal_rr_count na read_result
    list_of_vars = ideal_rr_count(read_result)
    list_of_vars.extend(number_of_insertions)

    Varijanta = ## TO DO

  return Varijanta

rr = '......,,,.....'
determine_variant(rr)
